# Act Log Analysis

In [1]:
# imports
import pickle

In [ ]:
# loading the act log
# change folder dir if this is no longer the demo
with open('../../topk/topk_act_base_hsm.pkl', 'rb') as f:
    topk_act_base_hsm1 = pickle.load(f)
with open('../../topk/topk_act_base_hsm1.pkl', 'rb') as f:
    topk_act_base_hsm1 = pickle.load(f)